In [3]:
cur_dir = os.getcwd()
os.chdir(os.path.join(cur_dir, 'data'))
!git clone https://gitlab.cs.washington.edu/ALGES/TACL2015.git
!git clone https://github.com/chaochun/nlu-asdiv-dataset.git
!git clone https://github.com/openai/grade-school-math.git
os.chdir(cur_dir)

fatal: destination path 'TACL2015' already exists and is not an empty directory.
fatal: destination path 'nlu-asdiv-dataset' already exists and is not an empty directory.
fatal: destination path 'grade-school-math' already exists and is not an empty directory.


In [20]:
import os
from xml.etree import ElementTree
import numpy as np
import torch
import json
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from io import StringIO
from contextlib import redirect_stdout
from termcolor import colored

In [21]:
import dataset_handler as dh

In [22]:
def preproc_gen_toks(gen_toks, input_len):
    list_out = []
    for gen_tok in gen_toks:
        last_tokens = gen_tok[input_len:]
        generated_text = tokenizer.decode(last_tokens)
        output = generated_text.split("\n\n")[0]
        list_out.append(output)
    return list_out

def pass_at_k(n, c, k):
    """
    :param n: total number of samples
    :param c: number of correct samples
    :param k: k in pass@$k$
    """
    if n - c < k:
        return 1.0
    return 1.0 - np.prod(1.0 - k / np.arange(n - c + 1, n + 1))

In [23]:
torch.manual_seed(0)
np.random.seed(8)

gptj_model = "EleutherAI/gpt-j-6B"
codeparrot_model = "lvwerra/codeparrot"

"""Load the priming text to add to the prompt and sample a question"""
priming_text_path = "data/priming_texts/gsm8k_onlyfullanswer.txt"
# priming_text = read_string_from_file("data/priming_texts/singleEq.txt")
# priming_text_path = "data/priming_texts/asdiv_genji.txt"

current_dataset = dh.init_dataset_from_name("gsm8k", primingtext_path=priming_text_path)

sample_q_list, sample_a_list = current_dataset.sample_n_for_prompting(1)

print(colored(sample_q_list[0], "blue"))
print(colored(sample_a_list[0], "green"))

Carol sells tickets for an exhibition. During three days she sold tickets worth $960. One ticket costs $4. How many tickets on average did she sell during one of these three days?
Carol sold 960 / 4 = <<960/4=240>>240 tickets during three days.
So in one day, she sold on average 240 / 3 = <<240/3=80>>80 tickets.
#### 80


In [7]:
"""GPT-J and codeparrot models run in HFTest venv"""
tokenizer = AutoTokenizer.from_pretrained(gptj_model)
model = AutoModelForCausalLM.from_pretrained(gptj_model).eval().cuda()

In [7]:
torch.manual_seed(42)
np.random.seed(42)

"""n = 4
k = 3"""
n = 3
k = 3

pass_k_list = []

for sample_q, sample_a in zip(sample_q_list, sample_a_list):
    prompt = f"{current_dataset.priming_text}\n\n#{sample_q}"
    #print(colored(f"\n\nPrompt:\n{prompt}", "green"))

    tokens = tokenizer(prompt, return_tensors="pt").input_ids
    generated_tokens = model.generate(
        tokens.long().cuda(),
        use_cache=True,
        do_sample=True,
        top_k=50,
        temperature=0.4,
        top_p=0.9,
        min_length=1,
        max_length=len(tokens[0]) + 100,
        num_return_sequences=n,
        pad_token_id=tokenizer.eos_token_id,
    )

    list_outputs = preproc_gen_toks(generated_tokens, len(tokens[0]))

    is_correct_list = [current_dataset.verify_pred_from_output(output, sample_q, sample_a) for output in list_outputs]

    c = is_correct_list.count(True)

    pass_k = pass_at_k(n, c, k)
    pass_k_list.append(pass_k)

 bricks are missing and the total cost of the wall is $<<220*0.5=110>>110
111111111111111.0
220.0
111111111111111.0
220.0
111111111111111.0
220.0


In [27]:
print(f"Pass@{k} = {np.mean(np.array(pass_k_list))}")

Pass@3 = 0.0
